In [1]:
# IMPORTS
import pandas as pd
import math
import os.path
import time
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

In [2]:
### API
binance_api_key =  #Enter your own API-key here
binance_api_secret = #Enter your own API-secret here

In [3]:
### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)

In [4]:
### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    return old, new

In [5]:
def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

In [6]:
data = get_all_binance("BTCUSDT", "1m", save=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13568\3465774344.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


All caught up..!


In [7]:
data

,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183,1502942459999,7564.906851,3,0.075183,320.390851,0
2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.0,1502942519999,0.0,0,0.0,0.0,0
2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074,1502942579999,1117.542921,2,0.261074,1117.542921,0
2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008,1502942639999,51.171852,3,0.012008,51.171852,0
2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796,1502942699999,599.999338,1,0.140796,599.999338,0
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-13 16:46:00,25848.59000000,25852.45000000,25843.93000000,25850.01000000,87.55776000,1686674819999,2263217.92826460,798,61.60338000,1592315.52380700,0
2023-06-13 16:47:00,25850.00000000,25866.98000000,25850.00000000,25866.97000000,50.89010000,1686674879999,1315885.29798440,898,39.49009000,1021080.61296140,0
2023-06-13 16:48:00,25866.98000000,25866.98000000,25855.67000000,25863.11000000,53.08733000,1686674939999,1372876.36503830,623,8.03349000,207753.86013710,0


In [8]:
len(data)

3053581

In [9]:
data.to_csv('Binance_BTCUSDT_1min.csv', index=True)

In [10]:
pd.read_csv("Binance_BTCUSDT_1min.csv")

,timestamp,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
0,2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183,1502942459999,7.564907e+03,3,0.075183,3.203909e+02,0
1,2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000,1502942519999,0.000000e+00,0,0.000000,0.000000e+00,0
2,2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074,1502942579999,1.117543e+03,2,0.261074,1.117543e+03,0
3,2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008,1502942639999,5.117185e+01,3,0.012008,5.117185e+01,0
4,2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796,1502942699999,5.999993e+02,1,0.140796,5.999993e+02,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3053576,2023-06-13 16:46:00,25848.59,25852.45,25843.93,25850.01,87.557760,1686674819999,2.263218e+06,798,61.603380,1.592316e+06,0
3053577,2023-06-13 16:47:00,25850.00,25866.98,25850.00,25866.97,50.890100,1686674879999,1.315885e+06,898,39.490090,1.021081e+06,0
3053578,2023-06-13 16:48:00,25866.98,25866.98,25855.67,25863.11,53.087330,1686674939999,1.372876e+06,623,8.033490,2.077539e+05,0
3053579,2023-06-13 16:49:00,25863.12,25864.17,25855.76,25864.17,12.433100,1686674999999,3.215178e+05,559,7.130000,1.843750e+05,0
